In [1]:
from datetime import date
import pandas as pd
import pyodbc
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
username = 'vigrose'
password_edw = 'Gryffindor~10946'

In [5]:
ins = pd.read_csv('../../Data/AMAIA/Newer_Insurance_2021-01-19.csv')
far = pd.read_csv('../../Data/AMAIA/Farther_Insurance_2021-01-19.csv')
farther = pd.read_csv('../../Data/AMAIA/Extra_Far_Insurance_2021-01-19.csv')
parties = tuple(ins['PARTY_ID'])

In [6]:
sql_query = \
        f'''
        SELECT DISTINCT
        A.PARTY_ID,
        A.POST_CD_ID,
        P.ADDR_1,
        P.ADDR_2,
        P.ADDR_3,
        P.CITY,
        S.SRC_STATE_CD AS STATE_CD,
        P.SRC_POST_KEY,
        P.POST_CD AS ZIP,
        A.FROM_DT,
        P.SRC_POST_KEY AS POST_KEY,
        C.PURPOSE_TYPE_DESC,
        C.PURPOSE_TYPE_ID,
        C.PURPOSE_USG_DESC
        FROM
        AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
        WHERE
        A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
        AND
        C.PURPOSE_CAT_CD='A'
        AND
        A.POST_CD_ID = P.POST_CD_ID
        AND
        P.STATE_ID = S.STATE_ID
        AND
        A.PARTY_ID in {parties}
        AND
        A.THRU_DT IS NULL
        '''
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [10]:
usages = pd.read_sql(con=AMAEDW, sql=sql_query)

In [11]:
okay_usages = usages[usages.PURPOSE_USG_DESC.isin(['Preferred Office','Preferred Professional Mailing Address','Insurance'])==False]

In [14]:
open_usage.groupby('PURPOSE_USG_DESC').count()

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,SRC_POST_KEY,ZIP,FROM_DT,POST_KEY,PURPOSE_TYPE_DESC,PURPOSE_TYPE_ID
PURPOSE_USG_DESC,,,,,,,,,,,,,
AMA Web Site,57144,57144,57144,56084,54832,57144,57144,57144,57144,57144,57144,57144,57144
AMC Address.,2577,2577,2577,2538,2536,2577,2577,2577,2577,2577,2577,2577,2577
Advantage AM News,32,32,32,32,32,32,32,32,32,32,32,32,32
Advantage Archive Journal,47,47,47,47,47,47,47,47,47,47,47,47,47
Advantage Journal of the American Medical Association,98,98,98,98,98,98,98,98,98,98,98,98,98
Billing Address.,1033,1033,1033,1001,1001,1033,1033,1033,1033,1033,1033,1033,1033
Membership Address.,191,191,191,190,190,191,191,191,191,191,191,191,191
Membership Shipping.,3035,3035,3035,3004,3001,3035,3035,3035,3035,3035,3035,3035,3035
No Usage,562635,562635,562634,543395,516006,562634,562635,562635,559833,562635,562635,562635,562635


In [41]:
def get_other_usages():
    '''Get insurance data from EDW'''
    query = get_query()
    bad_usages = get_bad_usages()
    print('Getting usages...')
    with EDW() as edw:
        usage = edw.read(query)
        print('Filtering...')
    usage = usage[usage.PURPOSE_TYPE_DESC.isin(bad_usages) == False]
    usage = usage[usage.PURPOSE_USG_DESC != 'Preferred Professional Mailing Address']
    return usage

def get_col():
    '''Get column dict'''
    columns = {
        'ME':'me#',
        'SRC_POST_KEY':'comm_id',
        'ADDR_1_y':'addr_line_1',
        'ADDR_2_y':'addr_line_2',
        'ADDR_3_y':'addr_line_3',
        'CITY_y':'addr_city',
        'STATE_CD_y':'addr_state',
        'ZIP_y':'addr_zip'}
    return columns

def fix_me(me_list):
    '''Add leading zeroes to ME'''
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

def clean_file(all_that):
    '''Merge and clean data'''
#     all_that = pd.merge(insurance, usage, on=['POST_CD_ID', 'PARTY_ID'])
#     all_that = all_that[all_that.PURPOSE_USG_DESC != 'Insurance']
    nones = all_that.fillna('None')
    missing_addresses = all_that[(nones.ADDR_2_y == 'None')&
                                 (nones.ADDR_1_y == 'None')&
                                 (nones.ADDR_3_y != 'None')]
    missing_addresses['ADDR_1_y'] = missing_addresses.ADDR_3_y
    all_that = all_that[(nones.ADDR_2_y != 'None')&(nones.ADDR_1_y != 'None')]
    all_that = all_that[nones.STATE_CD_y != 'None']
    all_that = pd.concat([all_that, missing_addresses])
    all_that = all_that.drop_duplicates(['POST_CD_ID', 'PARTY_ID'])
    all_that['usage'] = 'PP'
    all_that['load_type_ind'] = 'R'
    all_that['source'] = 'INS-USG'
    all_that['addr_type'] = 'N'
    all_that['source_dtm'] = str(date.today())
    col = get_col()
    all_that = all_that[['ME',
                         'SRC_POST_KEY',
                         'usage',
                         'load_type_ind',
                         'addr_type',
                         'ADDR_1_y',
                         'ADDR_2_y',
                         'ADDR_3_y',
                         'CITY_y',
                         'STATE_CD_y',
                         'ZIP_y',
                         'source',
                         'source_dtm',
                         ]].rename(columns=col)
    all_that['me#'] = fix_me(all_that['me#'])
    return all_that

def extra_file(insurance, usage):
    '''maybe'''
    print('Merging...')
    all_that = pd.merge(insurance, usage, on=['POST_CD_ID', 'PARTY_ID'])
    all_that = all_that[all_that.PURPOSE_USG_DESC != 'Insurance']
    all_that = all_that.drop_duplicates(['POST_CD_ID', 'PARTY_ID'])
    col = get_col()
    all_that = all_that[['ME',
                         'SRC_POST_KEY',
                         'ADDR_1_y',
                         'ADDR_2_y',
                         'ADDR_3_y',
                         'CITY_y',
                         'STATE_CD_y',
                         'ZIP_y',
                         'PURPOSE_USG_DESC',
                         'PURPOSE_TYPE_DESC'
                         ]].rename(columns=col)
    all_that['me#'] = fix_me(all_that['me#'])
    return all_that

def create_file(insurance):
    '''Create final file'''
    usage = get_other_usages()
    final_file = clean_file(insurance, usage)
    return final_file

def get_extra_file():
    '''ugh'''
    usage = get_other_usages()
    insurance = pd.read_csv('C:/Users/vigrose/Data/AMAIA/Insurance_2020-12-15.csv')
    final_file = extra_file(insurance, usage)
    final_file.to_csv('C:/Users/vigrose/Data/AMAIA/Insurance_Source_2020-11-25.csv', index=False)

In [4]:
ins = pd.read_csv('../../Data/AMAIA/Insurance_2020-12-15.csv')

In [17]:
bad_usages = ['License - California',
                  'License - Delaware',
                  'License - Montana',
                  'License - Nebraska',
                  'License - New Jersey',
                  'License - New York',
                  'License - Pennsylvania',
                  'License - Puerto Rico',
                  'License - Minnesota',
                  'License - Utah',
                  'License - Washington']
allowed_usage = okay_usages[okay_usages.PURPOSE_TYPE_DESC.isin(bad_usages) == False]

In [13]:
open_usage = usages[usages.PURPOSE_TYPE_ID.isin([317,318,1,2,269,281])==False]

In [45]:
with_usage = pd.merge(open_usage, ins, on=['PARTY_ID','STATE_CD'], suffixes=('_OTHER','_AMAIA'))
fuzzes = []
for row in with_usage.itertuples():
    fuzzes.append(fuzz.ratio(row.ADDR_1_OTHER.upper(),row.ADDR_1_AMAIA))

In [47]:
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [32]:
def count_sourced(df):
    fuzzes = []
    with_usage = pd.merge(okay_usages, df, on=['PARTY_ID','STATE_CD'], suffixes=('_OTHER','_AMAIA'))
    for row in with_usage.itertuples():
        fuzzes.append(fuzz.ratio(row.ADDR_1_OTHER.upper(),row.ADDR_1_AMAIA))
    with_usage['SCORE']=fuzzes
    with_usage = with_usage[with_usage.SCORE>71].drop_duplicates('POST_CD_ID_AMAIA')
    print(len(with_usage))
    allowed = with_usage[with_usage.PURPOSE_TYPE_DESC.isin(bad_usages)==False]
    print(len(allowed))
    return allowed

In [35]:
allowed_farther = count_sourced(farther)

597
467


In [31]:
allowed_usage.groupby('PURPOSE_TYPE_ID').count()

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,SRC_POST_KEY,ZIP,FROM_DT,POST_KEY,PURPOSE_TYPE_DESC,PURPOSE_USG_DESC
PURPOSE_TYPE_ID,,,,,,,,,,,,,
261,1,1,1,1,1,1,1,1,1,1,1,1,1
263,14,14,14,14,14,14,14,14,14,14,14,14,14
264,14,14,14,14,14,14,14,14,14,14,14,14,14
265,38,38,38,38,38,38,38,38,38,38,38,38,38
284,84,84,84,83,83,84,84,84,84,84,84,84,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,13,13,13,11,11,13,13,13,13,13,13,13,13
543,1681,1681,1681,1678,1677,1681,1681,1681,1681,1681,1681,1681,1681
544,1070,1070,1070,1048,1048,1070,1070,1070,1070,1070,1070,1070,1070


In [12]:
allowed.to_csv('../../Data/AMAIA/AMAIA_LOAD_NEW.csv', index=False)

In [3]:
newer = pd.read_csv('../../Data/AMAIA/Extra_Far_Insurance_2020-12-16.csv')
parties = tuple(newer['PARTY_ID'])

In [51]:
with_usage[['ADDR_1_OTHER','ADDR_1_AMAIA']]

,ADDR_1_OTHER,ADDR_1_AMAIA
0,1375 Hospital Dr,3082 YACHTSMAN DR.
1,135 Cannon St,3082 YACHTSMAN DR.
2,1375 Hospital Dr,3082 YACHTSMAN DR.
3,1375 Hospital Dr,3082 YACHTSMAN DR.
4,3082 Yachtsman Dr,3082 YACHTSMAN DR.
...,...,...
1761,6282 Linton Blvd,22015 ALTONA DRIVE
1762,1 Cliff Swallow Dr,1 CLIFFSWALLOW DRIVE
1763,307 Val Dr,1 CLIFFSWALLOW DRIVE
1764,1001 Main St,87 DARWIN DR.


In [54]:
with_usage['SCORE']=fuzzes

In [55]:
with_usage.sort_values('SCORE')

,PARTY_ID,POST_CD_ID_OTHER,ADDR_1_OTHER,ADDR_2_OTHER,ADDR_3_OTHER,CITY_OTHER,STATE_CD,SRC_POST_KEY,ZIP_OTHER,FROM_DT_OTHER,...,CITY_POLO,STATE_CD_POLO,ZIP_POLO,FROM_DT_POLO,POST_KEY_AMAIA,KEEP,NEW,POLO_DISTANCE,PPMA_DISTANCE,SCORE
709,2165266,23710874,,3135 3135 Zuni Street,,Denver,CO,19228345.0,80211,2016-10-19 21:40:26.901595,...,Columbia,MO,65201,2014-05-22 21:05:31.230726,15935587.0,True,True,681.182575,660.209400,0
777,2844925,26036472,,244 176 Th Ave SE Apt #302,,Mercer Island,WA,20242786.0,98040,2016-10-24 08:41:23.507739,...,Houston,TX,77057,2014-08-28 21:57:27.538140,14515141.0,True,True,1879.626634,1879.626634,0
1749,2253601,17049012,,Ccme Box F8,,Cripple Creek,CO,8977692.0,80813,2020-11-03 02:34:56.000000,...,Phoenix,AZ,85004,2011-10-27 21:48:06.584763,14762738.0,True,True,587.729349,584.326189,0
383,4573084,18162750,1647 Avocado Dr,,,Vista,CA,10574295.0,92083,2016-10-19 05:18:28.712984,...,Stuart,FL,34996,2016-11-03 21:30:51.042232,15037151.0,True,True,2232.830273,2231.763830,5
1129,5016465,29057778,5648 Locke Ln,None,None,Houston,TX,22633021.0,77056,2018-11-08 01:04:22.000000,...,Lansing,MI,48912,2012-03-08 22:17:26.421346,6625846.0,True,True,1124.323020,1123.134776,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,6010802,30024544,2506 Shepherdwood Ln,Tennessee,,Germantown,TN,23712915.0,38138,2019-07-30 21:13:55.905573,...,Chicago,IL,60616,2016-09-15 22:40:44.428351,14229644.0,True,True,481.229862,481.229862,100
1312,6010802,30024544,2506 Shepherdwood Ln,Tennessee,,Germantown,TN,23712915.0,38138,2019-07-30 21:13:55.908413,...,Chicago,IL,60616,2016-09-15 22:40:44.428351,14229644.0,True,True,481.229862,481.229862,100
731,3582200,24129611,1000 Gloucester Ct,,,Chapel Hill,NC,19572673.0,27516,2016-10-24 07:32:15.363342,...,Chattanooga,TN,37403,2018-09-14 20:55:22.313633,20461410.0,True,True,353.736352,353.736352,100
1653,2741881,20451592,6296 Trotter Rd,,,Clarksville,MD,13890807.0,21029,2020-04-15 01:40:19.000000,...,Geneva,IL,60134,2011-07-13 01:45:08.903449,14577366.0,True,True,623.770851,623.770851,100


In [56]:
bad_usages = ['License - California',
                  'License - Delaware',
                  'License - Montana',
                  'License - Nebraska',
                  'License - New Jersey',
                  'License - New York',
                  'License - Pennsylvania',
                  'License - Puerto Rico',
                  'License - Minnesota',
                  'License - Utah',
                  'License - Washington']
allowed = with_usage[with_usage.PURPOSE_TYPE_DESC.isin(bad_usages)==False]
ALLOWED = allowed[allowed.SCORE>71].drop_duplicates('POST_CD_ID_AMAIA')

In [60]:
allowed = with_usage[with_usage.PURPOSE_TYPE_DESC.isin(bad_usages)==False]

In [59]:
len(with_usage)

1766

In [64]:
allowed.sort_values('SCORE', ascending=False).to_csv('../../Data/AMAIA/Scoring.csv', index=False)

In [67]:
ALLOWED = allowed[allowed.SCORE>71].drop_duplicates('POST_CD_ID_AMAIA')

In [70]:
ALLOWED['FROM_DT_AMAIA'] = pd.to_datetime(ALLOWED.FROM_DT_AMAIA)

In [72]:
ALLOWED['AMAIA_YEAR'] = [x.year for x in ALLOWED.FROM_DT_AMAIA]

In [73]:
ALLOWED['AMAIA_MONTH'] = [x.month for x in ALLOWED.FROM_DT_AMAIA]

In [82]:
ALLOWED.groupby(['AMAIA_YEAR']).count()[['PARTY_ID']]

,PARTY_ID
AMAIA_YEAR,
2014,14
2015,5
2016,3
2017,6
2018,11
2019,29
2020,248


In [78]:
len(ins)

1211

In [81]:
len(ALLOWED)

316

In [84]:
ins['FROM_DT'] = pd.to_datetime(ins.FROM_DT)
ins['AMAIA_YEAR'] = [x.year for x in ins.FROM_DT]
ins.groupby(['AMAIA_YEAR']).count()[['PARTY_ID']]

,PARTY_ID
AMAIA_YEAR,
2014,184
2015,46
2016,43
2017,29
2018,37
2019,96
2020,776


In [85]:
ALLOWED

,PARTY_ID,POST_CD_ID_OTHER,ADDR_1_OTHER,ADDR_2_OTHER,ADDR_3_OTHER,CITY_OTHER,STATE_CD,SRC_POST_KEY,ZIP_OTHER,FROM_DT_OTHER,...,ZIP_POLO,FROM_DT_POLO,POST_KEY_AMAIA,KEEP,NEW,POLO_DISTANCE,PPMA_DISTANCE,SCORE,AMAIA_YEAR,AMAIA_MONTH
4,2495429,31069277,3082 Yachtsman Dr,,,Mt Pleasant,SC,24699474.0,29466,2020-04-05 11:28:55.000000,...,7631,2018-02-09 20:41:35.303772,11826980.0,True,True,640.962597,640.962597,97,2020,8
10,6159440,15752915,10815 W 104th St,,,Overland Park,KS,1265183.0,66214,2016-10-17 14:42:11.516447,...,21215,2015-08-13 21:51:38.120692,20149855.0,True,True,965.378235,965.378235,94,2020,8
16,2108206,30378781,881 NE Locksley Dr,None,None,Bend,OR,24101905.0,97701,2019-11-16 03:59:48.000000,...,96814,2020-09-20 21:08:17.514750,25260422.0,True,True,2610.724033,2610.724033,92,2020,12
21,2116371,14230876,322 Garcia St,,,Santa Fe,NM,1376617.0,87501,2016-10-17 14:44:37.367693,...,92121,2011-10-27 21:45:13.540679,14766137.0,True,True,668.766248,668.766248,96,2020,5
29,1836641,29505516,2300 NW Corporate Blvd,None,None,Boca Raton,FL,23249961.0,33431,2019-02-22 00:27:45.000000,...,38654,2018-06-29 20:44:42.107730,22096597.0,True,True,827.594453,827.594453,81,2020,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,1868100,31742637,2011 W Danforth Rd,# 514,,Edmond,OK,25331743.0,73003,2020-11-04 00:48:56.000000,...,95630,2014-10-09 20:59:17.314774,14526213.0,True,True,1314.803677,1314.803677,92,2020,12
1756,2581697,21087407,101 Pipe Creek Ln,,,Georgetown,TX,14501673.0,78633,2020-11-17 11:40:52.000000,...,99603,2005-12-16 21:47:00.979919,7646838.0,True,True,3150.671311,3150.671311,100,2014,6
1757,2555337,25500807,1423 Nob Hill Ave N,,,Seattle,WA,19818380.0,98109,2020-11-17 11:40:52.000000,...,99508,2007-12-19 22:12:29.218930,8542845.0,True,True,1429.607240,1429.607240,100,2014,6
1759,4623497,31851515,116 E Brittany Dr,,,Arlington Hts,IL,25484493.0,60004,2020-11-21 04:10:26.000000,...,11212,2016-12-07 21:31:42.815874,20857603.0,True,True,735.071523,735.071523,89,2020,9


In [93]:
allowed[allowed.SCORE>71].groupby('PURPOSE_TYPE_DESC').count().sort_values('PARTY_ID', ascending=False)

,PARTY_ID,POST_CD_ID_OTHER,ADDR_1_OTHER,ADDR_2_OTHER,ADDR_3_OTHER,CITY_OTHER,STATE_CD,SRC_POST_KEY,ZIP_OTHER,FROM_DT_OTHER,...,CITY_POLO,STATE_CD_POLO,ZIP_POLO,FROM_DT_POLO,POST_KEY_AMAIA,KEEP,NEW,POLO_DISTANCE,PPMA_DISTANCE,SCORE
PURPOSE_TYPE_DESC,,,,,,,,,,,,,,,,,,,,,
License - Florida,79,79,79,55,54,79,79,79,79,79,...,79,79,79,79,79,79,79,79,79,79
License - Texas,52,52,52,38,38,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
DEA,34,34,34,25,20,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
License - Ohio,33,33,33,27,26,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
License - Illinois,30,30,30,26,26,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
License - Indiana,26,26,26,18,18,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
Home,25,25,25,23,23,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
Unknown,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
Permanent (Parents),17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17


In [92]:
len(allowed[allowed.SCORE>71])

524

In [94]:
allo = allowed[allowed.SCORE>71]

In [ ]:
for row in allo.itertuples()

In [95]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20201205.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [96]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [100]:
ppd[['MAILING_NAME','PHONE', 'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2']]

,MAILING_NAME,PHONE,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2
0,MICHAEL J STANFORD MD,nan,NaN,NaN
1,MERI K MARTIN MD,nan,NaN,8101 BROWNLEIGH DR
2,MARK T JACKSON MD,8282544337.0,MOUNTAIN AREA PEDIATRIC ASSOC,500 CENTREPARK DR
3,NADHIWAN S MAISIAK MD,3138732225.0,NaN,14502 W MEEKER BLVD
4,RAJIV B GALA MD,5048429618.0,OCHSNER MEDICAL CENTER,4429 CLARA ST STE 400
...,...,...,...,...
1281357,ROBERT SAKULANDA MD,nan,NaN,NaN
1281358,KAMISHELE MAILA MD,4199965832.0,ST RITA'S MEDICAL CENTER,730 W MARKET ST
1281359,MWIZA GAUSI MD,4347992100.0,DANVILLE REGIONAL MEDICAL,142 S MAIN ST
1281360,MASIKU A MDALA-GAUSI MD,3367135215.0,NaN,1 MEDICAL CENTER BLVD


In [98]:
ppd['PHONE']=ppd.TELEPHONE_NUMBER.astype(str)

In [101]:
def clean_zipcode(zipcode):
    '''Format zipcode'''
    zipcode = str(zipcode)
    zipcode = zipcode.replace(" ", "")
    zipcode = zipcode.replace(".0", "")
    if len(zipcode) == 3:
        zipcode = "00"+zipcode
    if len(zipcode) == 4:
        zipcode = "0"+zipcode
    else:
        zipcode = zipcode[0:5]
    return zipcode

In [102]:
clean_zipcode('None')

'0None'

In [103]:
ppd['POLO_MAILING_LINE_1'].fillna('boop')

0                                   boop
1                                   boop
2          MOUNTAIN AREA PEDIATRIC ASSOC
3                                   boop
4                 OCHSNER MEDICAL CENTER
                       ...              
1281357                             boop
1281358         ST RITA'S MEDICAL CENTER
1281359        DANVILLE REGIONAL MEDICAL
1281360                             boop
1281361                             boop
Name: POLO_MAILING_LINE_1, Length: 1281362, dtype: object

In [104]:
from datetime import date

In [106]:
date(2005, 1,1)

datetime.date(2005, 1, 1)

In [107]:
import pgeocode
def get_zip_distance(zip1, zip2, dist):
    '''Get distances between zipcodes'''
    zip1 = clean_zipcode(zip1)
    zip2 = clean_zipcode(zip2)
    distance = dist.query_postal_code(zip1, zip2)*0.621371
    if distance
    return distance

In [108]:
dist = pgeocode.GeoDistance('US')

In [110]:
get_zip_distance(60608, 'a', dist)

nan

In [112]:
lol = pd.read_csv("../../Data/AMAIA/Newer_Insurance_2020-12-16.csv")

In [128]:
def sum_it_up(df):
    '''Remove matching addresses'''
    print("Removing matching addresses...")
    df = df.fillna('None')
    count =0 
    for row in df.itertuples():
        if row.POST_CD_ID_POLO != row.POST_CD_ID and row.POST_CD_ID != row.POST_CD_ID_PPMA:
            if row.ZIP != row.ZIP_PPMA and row.ZIP != row.ZIP_POLO: 
                if row.STATE_CD != row.STATE_CD_PPMA and row.STATE_CD != row.STATE_CD_POLO:
                    count+= 1
    return count

In [129]:
sum_it_up(lol)

Removing matching addresses...


2824

In [116]:
len(lol)

103114

In [ ]:
def sum_it_up(df):
    '''Remove matching addresses'''
    keep_list = []
    print("Removing matching addresses...")
    df = df.fillna('None')
    for row in df.itertuples():
        keep = False
        if row.POST_CD_ID_POLO != row.POST_CD_ID and row.POST_CD_ID != row.POST_CD_ID_PPMA:
            if row.STATE_CD != row.STATE_CD_PPMA and row.STATE_CD != row.STATE_CD_POLO:
                if row.CITY != row.CITY_PPMA and row.CITY != row.CITY_POLO:
                    if row.ZIP != row.ZIP_PPMA and row.ZIP != row.ZIP_POLO:
                        if row.ADDR_1 != row.ADDR_1_PPMA and row.ADDR_1 != row.ADDR_1_POLO:
                            keep = True
        keep_list.append(keep)
    df['NEW'] = keep_list
    df = df[df.NEW == True]
    return df

In [121]:
lol.columns

Index(['PARTY_ID', 'ME', 'POST_CD_ID', 'FROM_DT', 'ADDR_TYPE', 'ADDR_1',
       'ADDR_2', 'ADDR_3', 'CITY', 'STATE_CD', 'ZIP', 'POST_CD_ID_PPMA',
       'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA', 'CITY_PPMA',
       'STATE_CD_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POST_CD_ID_POLO',
       'ADDR_1_POLO', 'ADDR_2_POLO', 'ADDR_3_POLO', 'CITY_POLO',
       'STATE_CD_POLO', 'ZIP_POLO', 'FROM_DT_POLO', 'POST_KEY', 'KEEP'],
      dtype='object')

In [159]:
def sum_it_up(df):
    '''Remove matching addresses'''
    keep_list = []
    print("Removing matching addresses...")
    df = df.fillna('None')
    for row in df.itertuples():
        keep = False
        if row.POST_CD_ID_POLO != row.POST_CD_ID and row.POST_CD_ID != row.POST_CD_ID_PPMA:
            if row.ZIP != row.ZIP_PPMA and row.ZIP != row.ZIP_POLO:
                keep = True
        keep_list.append(keep)
    df['NEW'] = keep_list
    df = df[df.NEW == True]
    return df

In [160]:
beep = sum_it_up(newer)

Removing matching addresses...


In [131]:
all_this = pd.read_csv('../../Data/AMAIA/Unfiltered_Insurance_2020-12-16.csv')

In [134]:
parties = tuple(all_this['PARTY_ID'])

In [133]:
len(parties)

1211

In [162]:
import snscrape.modules.twitter as sntwitter

maxTweets = 10

for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:@TwitterSupport').get_items()) :
        if i > maxTweets :
            break  
        print(f"the date is {tweet.date}")  
        print(f"the user name is {tweet.user.username}")
        print(f"the tweet content is {tweet.content}")
        print(f"the tweet rendered content is {tweet.renderedContent}")
        print(f"the outlinks are {tweet.outlinks}")  
        print(f"the tco outlinks are {tweet.tcooutlinks}") 
        print(f"the url is {tweet.url}")
        print(f"the retweeted tweet is  {tweet.retweetedTweet}")        
        print(f"the quoted tweet is  {tweet.quotedTweet}") 

ModuleNotFoundError: No module named 'snscrape'

In [36]:
allowed_far.columns

Index(['PARTY_ID', 'POST_CD_ID_OTHER', 'ADDR_1_OTHER', 'ADDR_2_OTHER',
       'ADDR_3_OTHER', 'CITY_OTHER', 'STATE_CD', 'SRC_POST_KEY', 'ZIP_OTHER',
       'FROM_DT_OTHER', 'POST_KEY_OTHER', 'PURPOSE_TYPE_DESC',
       'PURPOSE_TYPE_ID', 'PURPOSE_USG_DESC', 'ME', 'POST_CD_ID_AMAIA',
       'FROM_DT_AMAIA', 'ADDR_TYPE', 'ADDR_1_AMAIA', 'ADDR_2_AMAIA',
       'ADDR_3_AMAIA', 'CITY_AMAIA', 'ZIP_AMAIA', 'POST_CD_ID_PPMA',
       'ADDR_1_PPMA', 'ADDR_2_PPMA', 'ADDR_3_PPMA', 'CITY_PPMA',
       'STATE_CD_PPMA', 'ZIP_PPMA', 'FROM_DT_PPMA', 'POST_CD_ID_POLO',
       'ADDR_1_POLO', 'ADDR_2_POLO', 'ADDR_3_POLO', 'CITY_POLO',
       'STATE_CD_POLO', 'ZIP_POLO', 'FROM_DT_POLO', 'POST_KEY_AMAIA', 'KEEP',
       'NEW', 'POLO_DISTANCE', 'PPMA_DISTANCE', 'SCORE'],
      dtype='object')

In [42]:
clean_file(allowed_farther)

AttributeError: 'DataFrame' object has no attribute 'ADDR_2_y'

In [43]:
allowed_far

,PARTY_ID,POST_CD_ID_OTHER,ADDR_1_OTHER,ADDR_2_OTHER,ADDR_3_OTHER,CITY_OTHER,STATE_CD,SRC_POST_KEY,ZIP_OTHER,FROM_DT_OTHER,...,CITY_POLO,STATE_CD_POLO,ZIP_POLO,FROM_DT_POLO,POST_KEY_AMAIA,KEEP,NEW,POLO_DISTANCE,PPMA_DISTANCE,SCORE
0,2474649,14398126,287 Connie Ave,,,Los Alamos,NM,6448543.0,87547,2006-03-01 23:37:44.138572,...,None,None,None,2005-01-01,None,True,True,999.000000,1612.236603,100
4,2201599,20931687,6331 E Placita Atiza,,,Tucson,AZ,14395735.0,85750,2019-09-19 21:09:45.677334,...,None,None,None,2005-01-01,None,True,True,999.000000,794.828424,98
9,1936285,11725834,3 Ridge Farm Rd,,,Burr Ridge,IL,878956.0,60527,2016-10-17 14:29:48.114323,...,None,None,None,2005-01-01,None,True,True,999.000000,1039.026261,100
17,2495429,31069277,3082 Yachtsman Dr,,,Mt Pleasant,SC,24699474.0,29466,2020-04-05 11:28:55.000000,...,Englewood,NJ,7631.0,2018-02-09 20:41:35.303772,11826980.0,True,True,640.962597,640.962597,97
18,1937642,14577673,328 Iron Horse Rd,,,Rocky Mount,NC,936993.0,27804,2016-10-17 14:31:59.582792,...,Easton,MD,21601.0,2018-05-11 20:49:27.396101,8320368.0,True,True,215.668273,215.668273,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,2581697,21087407,101 Pipe Creek Ln,,,Georgetown,TX,14501673.0,78633,2020-11-17 11:40:52.000000,...,Homer,AK,99603.0,2005-12-16 21:47:00.979919,7646838.0,True,True,3150.671311,3150.671311,100
2778,2555337,25500807,1423 Nob Hill Ave N,,,Seattle,WA,19818380.0,98109,2020-11-17 11:40:52.000000,...,Anchorage,AK,99508.0,2007-12-19 22:12:29.218930,8542845.0,True,True,1429.607240,1429.607240,100
2780,4623497,31851515,116 E Brittany Dr,,,Arlington Hts,IL,25484493.0,60004,2020-11-21 04:10:26.000000,...,Brooklyn,NY,11212.0,2016-12-07 21:31:42.815874,20857603.0,True,True,735.071523,735.071523,89
2787,4504809,31876343,8065 Station House Rd,,,Dayton,OH,25519775.0,45458,2020-12-05 01:27:37.000000,...,Washington,DC,20017.0,2012-11-29 22:37:44.918725,5916286.0,True,True,386.481835,386.481835,98


In [46]:
allowed_far[['PARTY_ID', 'POST_CD_ID_OTHER', 'ADDR_1_OTHER', 'ADDR_2_OTHER',
       'ADDR_3_OTHER', 'CITY_OTHER', 'STATE_CD', 'SRC_POST_KEY', 'ZIP_OTHER',
       'FROM_DT_OTHER', 'POST_KEY_OTHER', 'PURPOSE_TYPE_DESC',
       'PURPOSE_TYPE_ID', 'PURPOSE_USG_DESC', 'ME']].to_csv('../../Data/AMAIA/ALLOWED_FAR.csv', index=False)

In [47]:
allowed_farther[['PARTY_ID', 'POST_CD_ID_OTHER', 'ADDR_1_OTHER', 'ADDR_2_OTHER',
       'ADDR_3_OTHER', 'CITY_OTHER', 'STATE_CD', 'SRC_POST_KEY', 'ZIP_OTHER',
       'FROM_DT_OTHER', 'POST_KEY_OTHER', 'PURPOSE_TYPE_DESC',
       'PURPOSE_TYPE_ID', 'PURPOSE_USG_DESC', 'ME']].to_csv('../../Data/AMAIA/ALLOWED_FARTHER.csv', index=False)